In [1]:
import pdftotext
import pandas as pd
import numpy as np
import re
import os

#%% # snetiment analysis
import numpy as np
import pandas as pd

import re
import string 

from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define function for "Cleaning" and "participants list"

In [2]:
def cleaning_text(contents):
    # change the text input to df
    df = pd.DataFrame(contents)
    # remove the unnessary string
    df[0] = df[0].str.replace('\n','')
    df[0] = df[0].str.replace('Bloomberg Transcript','')
    df[0] = df[0].str.replace('\x0c\n','')
    df[0] = df[0].str.replace('FINAL','')
    df[0] = df[0].str.replace('A - ','')
    df[0] = df[0].str.replace('Q - ','')
    # using re to remove the unnessary string
    def drop_unnessary(x):
        page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
        BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
        Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
        Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
        Date = re.findall(r'Date:', x) # Date: 2015-03-10
        if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_unnessary(x))
    df = df[true_false]

    # drop the final page declaration
    df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
    df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
    df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
    df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
    df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
    df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
    df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
    df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
    df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
    # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  # we will need this to identify the last participant
    df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
    # ¬© could not be identified, would apply re
    def drop_Bloomberg_mark(x):
        Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
        if Bloomberg_mark == []:
            return True
        else:
            return False

    true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
    df = df[true_false]

    # drop the empthy row
    df = df[df[0] != '']
    df = df[df[0] != '']

    return df

def participants_list(df):
    # reset the index to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    #  'Company Participants' index
    Participant_start_index = df.index[df.iloc[:,0] == 'Company Participants'].tolist()
    #  'Other Participants' index
    Participant_middle_index = df.index[df.iloc[:,0] == 'Other Participants'].tolist()
    #  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
    Participant_end_index = df.index[df.iloc[:,0] == 'MANAGEMENT DISCUSSION SECTION' ].tolist()
    # try to find the 'MANAGEMENT DISCUSSION SECTION' or 'Presentation' index
    if Participant_end_index == []:
        Participant_end_index = df.index[df.iloc[:,0] == 'Presentation'].tolist()
        Participant_end_index = [Participant_end_index[-1]]
    # some transcript dont have 'Other Participants'
    if Participant_middle_index == []:
        Participant_middle_index = Participant_end_index
    print(Participant_start_index, Participant_middle_index, Participant_end_index)

    # make the list of company_paticipants 
    company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
    company_paticipants.drop(company_paticipants.index[company_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    company_paticipants = company_paticipants.values.tolist()
    # and other_participants
    other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
    other_paticipants.drop(other_paticipants.index[other_paticipants.iloc[:,0] == ''].tolist(), inplace=True)
    other_paticipants = other_paticipants.values.tolist()

    # after extract the paticipants, we can drop those information to make the transcript more clear
    df = df.reset_index(drop=True)
    df = df.drop(range(df.index[df.iloc[:,0] == 'Company Participants'].tolist()[0],df.index[df.iloc[:,0].isin(['MANAGEMENT DISCUSSION SECTION','Presentation'])].tolist()[0]+1))
    # drop the first row of the df
    df = df.reset_index(drop=True)
    df = df.iloc[1: , :]
    # reset the index again to make sure the index is continuous for better processing
    df = df.reset_index(drop=True)
    # # save to csv
    # df.to_csv('/Users/timliu/Desktop/output/df.csv')
    return df, company_paticipants, other_paticipants

In [3]:
path = "/Users/hienanh/Documents/GitHub/final_01/Transcript_test"
save_path = "/Users/hienanh/Documents/GitHub/final_01/Output"

In [4]:
# all files path
company_paths = []
sectors = os.listdir(path) 
for sector in sectors:
    # path to each sector files
    if sector != '.DS_Store':
        sector_path = path+"/"+sector
    
    # path to each company files
    companies = os.listdir(sector_path) 
    #print(companies)
    for company in companies:
        if company != '.DS_Store':
            company_path = sector_path+"/"+company
            company_paths.append(company_path)

# create dataframe
df = pd.DataFrame()
df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index']) # create a dataframe with 2500 rows
all_participants = []      
company_paticipants_list = []
other_paticipants_list = []

for single_path in company_paths:
    files = os.listdir(single_path)
    for file in files:
        print(file)
        if file.endswith(".pdf"):
            # print(file)
            # Load PDF
            with open(single_path+"/"+file, "rb") as f:
                pdf = pdftotext.PDF(f)
            # Save all text to a txt file.
            with open(save_path+"/"+file.replace(".pdf", ".txt"), "w") as f:
                f.write("\n\n".join(pdf))
            # open the text file
            with open(save_path+"/"+file.replace(".pdf", ".txt")) as f:
                contents = f.readlines()
                df_clean = cleaning_text(contents)
                # extract all the participants
                df_pure_text,company_paticipants,other_paticipants = participants_list(df_clean)
                all_participants.append(company_paticipants)
                all_participants.append(other_paticipants)
                # using the file name to set as the dataframe's column name
                # df[f"{files.index(file)}"] = df_clean
                df[f"{files[files.index(file)]}"] = df_pure_text
                df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na.head(5)

20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf
[1] [4] [12]
20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf


/Users/hienanh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[2] [8] [15]
20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf
[1] [3] [10]
20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf
[1] [4] [11]
20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf
[1] [4] [14]
20141119_Sompo_Holdings_Inc-_Earnings_Call_2014-11-24_FS000000002417374854.pdf
[1] [4] [11]
20151118_Sompo_Holdings_Inc-_Earnings_Call_2015-11-20_DN000000002241138647.pdf
[1] [3] [12]
20131119_Sompo_Holdings_Inc-_Earnings_Call_2013-11-25_DN000000002116348219.pdf
[1] [3] [10]
.DS_Store
20181119_Sompo_Holdings_Inc-_Earnings_Call_2018-11-19_SD000000002905610039.pdf
[1] [4] [12]
20151126_Sompo_Holdings_Inc-_Conf-Presentation_Call_2015-11-26_SD000000002889371132.pdf
[1] [4] [12]
20190528_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-5-28_SD000000002902938862.pdf
[2] [8] [16]
20150520_Sompo_Holdings_Inc-_Earnings_Call_2015-5-21_DN000000002212245169.pdf
[1] [3] [11]
20161005_Sompo_Holdings_Inc-_M-A_Call_2016-10-5_F

[1] [7] [12]
20170519_TOKIO_MARINE_HD-_Earnings_Call_2017-5-19_SD000000002893098131.pdf
[1] [4] [11]
20160212_TOKIO_MARINE_HD-_Earnings_Call_2016-2-12_FS000000002253111492.pdf
[1] [4] [9]
.DS_Store
20160809_TOKIO_MARINE_HD-_Earnings_Call_2016-8-9_FS000000002298049534.pdf
[1] [3] [11]
20170808_TOKIO_MARINE_HD-_Earnings_Call_2017-8-8_FS000000002365225099.pdf
[1] [4] [13]
20151118_TOKIO_MARINE_HD-_Earnings_Call_2015-11-18_FS000000002240862350.pdf
[1] [3] [11]
20210527_TOKIO_MARINE_HD-_Shareholder_Mtg_Call_2021-5-27_SD000000002958087539.pdf


/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/4065511328.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)
/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/4065511328.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{files[files.index(file)]}"] = df_pure_text


[1] [9] [18]
20120531_TOKIO_MARINE_HD-_Conf-Presentation_Call_2012-5-31_SD000000002867310091.pdf
[2] [9] [17]
20130213_TOKIO_MARINE_HD-_Earnings_Call_2013-2-13_DN000000002077439881.pdf
[1] [3] [8]
20150520_TOKIO_MARINE_HD-_Earnings_Call_2015-5-20_DN000000002212037068.pdf
[1] [3] [3]
20170519_TOKIO_MARINE_HD-_Earnings_Call_2017-5-19_FS000000002351735615.pdf
[1] [4] [11]
20191003_TOKIO_MARINE_HD-_M-A_Call_2019-10-3_SD000000002888014841.pdf
[2] [4] [4]
20190527_TOKIO_MARINE_HD-_Conf-Presentation_Call_2019-5-27_SD000000002902936847.pdf
[1] [11] [19]
20180214_TOKIO_MARINE_HD-_Earnings_Call_2018-2-14_FS000000002400295131.pdf
[1] [3] [7]
20131119_TOKIO_MARINE_HD-_Earnings_Call_2013-11-20_DN000000002115755239.pdf
[1] [5] [9]
20130809_TOKIO_MARINE_HD-_Earnings_Call_2013-8-12_DN000000002102494109.pdf
[1] [4] [11]
20200520_TOKIO_MARINE_HD-_Earnings_Call_2020-5-20_RT000000002841939564.pdf
[1] [3] [3]
20160520_TOKIO_MARINE_HD-_Earnings_Call_2016-5-20_FS000000002279195483.pdf
[1] [3] [10]
20171117_T

[1] [4] [11]
20110227_QBE_INSURANCE-_Earnings_Call_2011-2-27_SD000000002767279336.pdf
[1] [4] [15]
20181210_QBE_INSURANCE-_Guidance_Call_2018-12-10_SD000000002901842698.pdf
[3] [6] [17]
20210219_QBE_INSURANCE-_Earnings_Call_2021-2-18_DN000000002950935440.pdf
[1] [5] [15]
20180225_QBE_INSURANCE-_Earnings_Call_2018-2-25_SD000000002902635797.pdf
[1] [4] [18]
20150818_QBE_INSURANCE-_Earnings_Call_2015-8-17_FS000000002226588573.pdf
[1] [5] [14]
20121112_QBE_INSURANCE-_Guidance_Call_2012-11-12_SD000000002822570969.pdf
[1] [4] [16]
20180816_QBE_INSURANCE-_Earnings_Call_2018-8-15_DN000000002458573055.pdf
[1] [4] [10]
20180121_QBE_INSURANCE-_Guidance_Call_2018-1-23_FS000000002406211386.pdf
[1] [3] [13]
20170227_QBE_INSURANCE-_Earnings_Call_2017-2-26_FS000000002332481646.pdf
[1] [6] [17]
20190814_QBE_INSURANCE-_Earnings_Call_2019-8-14_DN000000002695062133.pdf
[1] [4] [10]
20110404_QBE_INSURANCE-_Shareholder_Mtg_Call_2011-4-7_RT000000002023727287.pdf
[1] [6] [8]
20120112_QBE_INSURANCE-_Earnings_C

[1] [8] [20]
20200227_SCOR_SE-_Earnings_Call_2020-2-27_RT000000002799748126.pdf
[1] [10] [17]
20170427_SCOR_SE-_Earnings_Call_2017-4-27_FS000000002465467570.pdf
[1] [8] [18]
20150506_SCOR_SE-_Earnings_Call_2015-5-6_FS000000002465467276.pdf
[1] [9] [18]
20211027_SCOR_SE-_Earnings_Call_2021-10-27_DN000000002966600307.pdf
[1] [10] [17]
20160427_SCOR_SE-_Earnings_Call_2016-4-27_FS000000002465467432.pdf
[1] [9] [20]
20141106_SCOR_SE-_Earnings_Call_2014-11-6_FS000000002464032501.pdf
[1] [8] [19]
20210728_SCOR_SE-_Earnings_Call_2021-7-28_DN000000002961207792.pdf
[1] [9] [18]
20180905_SCOR_SE-_Shareholder_Mtg_Call_2018-9-5_SD000000002902712952.pdf
[1] [19] [28]
20150305_SCOR_SE-_Earnings_Call_2015-3-5_FS000000002464032531.pdf
[1] [9] [18]
.DS_Store
20210209_SCOR_SE-_M-A_Call_2021-2-9_DN000000002949795370.pdf
[1] [5] [12]
20190220_SCOR_SE-_Earnings_Call_2019-2-20_FS000000002590782428.pdf
[1] [10] [18]
20160727_SCOR_SE-_Earnings_Call_2016-7-27_FS000000002465467474.pdf
[1] [9] [21]
20170222_SCOR_

[1] [5] [17]
20150507_Munich_Re-_Earnings_Call_2015-5-7_FS000000002450434991.pdf
[1] [6] [17]
20181107_Munich_Re-_Earnings_Call_2018-11-7_FS000000002510108193.pdf
[1] [5] [19]
20180315_Munich_Re-_Earnings_Call_2018-3-15_DN000000002409987736.pdf
[1] [7] [10]
20190807_Munich_Re-_Earnings_Call_2019-8-7_DN000000002686215020.pdf
[1] [6] [19]
20201105_Munich_Re-_Earnings_Call_2020-11-5_DN000000002928821379.pdf
[1] [4] [13]
20200228_Munich_Re-_Earnings_Call_2020-2-28_RT000000002825328883.pdf
[1] [7] [20]
20180315_Munich_Re-_Earnings_Call_2018-3-15_FS000000002411211650.pdf
[1] [8] [23]
20180808_Munich_Re-_Earnings_Call_2018-8-8_FS000000002455566648.pdf
[1] [5] [19]
20160809_Munich_Re-_Earnings_Call_2016-8-9_FS000000002298114256.pdf
[1] [6] [19]
20211109_Munich_Re-_Earnings_Call_2021-11-9_DN000000002967620660.pdf
[1] [5] [14]
20150205_Munich_Re-_Earnings_Call_2015-2-5_FS000000002321980336.pdf
[1] [4] [8]
20210225_Munich_Re-_Earnings_Call_2021-2-25_DN000000002951500442.pdf
[1] [5] [15]
20210810_

,20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf,20141119_Sompo_Holdings_Inc-_Earnings_Call_2014-11-24_FS000000002417374854.pdf,20151118_Sompo_Holdings_Inc-_Earnings_Call_2015-11-20_DN000000002241138647.pdf,20131119_Sompo_Holdings_Inc-_Earnings_Call_2013-11-25_DN000000002116348219.pdf,20181119_Sompo_Holdings_Inc-_Earnings_Call_2018-11-19_SD000000002905610039.pdf,20151126_Sompo_Holdings_Inc-_Conf-Presentation_Call_2015-11-26_SD000000002889371132.pdf,...,20210810_Munich_Re-_Earnings_Call_2021-8-10_DN000000002962247258.pdf,20201208_Munich_Re-_Shareholder_Mtg_Call_2020-12-8_DN000000002944723654.pdf,20200507_Munich_Re-_Earnings_Call_2020-5-7_RT000000002834670399.pdf,20171109_Munich_Re-_Earnings_Call_2017-11-9_FS000000002383615110.pdf,20170509_Munich_Re-_Earnings_Call_2017-5-9_FS000000002450443745.pdf,20170315_Munich_Re-_Earnings_Call_2017-3-15_FS000000002336461963.pdf,20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf
0,Shinichi Hara,Presentation,Unverified Participant,Osamu Nose,Shinichi Hara,Yukinori Kuroda,Yukinori Kuroda,Shinichi Hara,Yukinori Kuroda,Kengo Sakurada,...,Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Operator,Muenchen Investor Day,Christian Becker-Hussong,Joerg Schneider
1,I am Hara from NKSJ Holdings. Thank you very m...,Company Participants,I thank all of you for gathering here today fo...,"Hello. I am Osamu Nose, Head of Investor Relat...",Good evening. This is Shinichi Hara of NKSJ Ho...,"Hello, everyone. I'm Kuroda of Investor Relati...",So this is Kuroda from Sompo Japan Nipponkoa H...,I'm Hara from NK Share Holdings. Thank you ver...,I'm Kuroda with Sompo Holdings. Thank you very...,"As introduced, I am Sakurada, the Group CEO. F...",...,"Good day, and welcome to the Munich Re Half Ye...","Good afternoon, ladies and gentlemen and warm ...","Good day, and welcome to the Muenchener Quarte...",Thanks for joining us for our Q3 earnings call...,"Good day, ladies and gentlemen, and welcome to...","Ladies and gentlemen, welcome to the Munich Re...","Good day, ladies and gentlemen. Welcome to the...",Christian Becker-Hussong,Thank you. Ladies and gentlemen a warm welcome...,"Ladies and gentlemen, welcome to the presentat..."
2,sure you're tired by now at this late hour but...,"Kengo Sakurada, President, Group CEO, Chairman...",are as we have announced on the 18th. Therefor...,Holdings. Thank you for participating in our c...,busy schedule to attend this conference call. ...,Thank you very much for joining us today in th...,participating in today's conference call.,"call for the first half results. Today, I'll c...",schedule to this conference call. I would like...,meeting despite your busy schedule. It seems t...,...,Conference Call. Today's conference is being r...,Investor Day on Munich Re Group's Ambition 202...,Results Conference Call. Today's conference is...,"very straightforward, after my short introduct...",Q1 Conference Call. Today's conference is bein...,room. And of course also a very warm welcome t...,Statement Conference Call. Today's conference ...,"Good morning, everyone. Welcome to our Investo...",Financial Results 2018 and Strategic Outlook. ...,Schneider and I am the CFO of Munich Re Group.
3,What I am going to explain today is only the f...,"Masahiro Hamada, Group CFO, Group CIO, Executi...","for myself today, I will be mainly talking abo...",I would like to go through our

In [5]:
def clean_participants_list(def_participants):
    # get the value inside the def_participants 
    def_participants = [item for sublist in def_participants for item in sublist]
    def_participants = [i[0] for i in def_participants]
    # print(def_participants)
    # %%
    # exclude the title of the participants, i.e.'Roland Vogel, CFO' to 'Roland Vogel" by using re
    def_participants = [re.sub(r'\,.*', '', participant) for participant in def_participants]
    # exclude the 'Property & Casualty Reinsurance'
    def_participants = [re.sub(r'Property & Casualty Reinsurance', '', participant) for participant in def_participants]
    # exclude the '[0682QB-E Ulrich Wallin]'
    def_participants = [re.sub(r'\[0682QB-E Ulrich Wallin\]', '', participant) for participant in def_participants]
    # drop duplicated participants
    # def_participants = [i[0] for i in def_participants]
    # drop the empty string
    def_participants = [participant for participant in def_participants if participant != '']
    # remove the sapce in the string
    def_participants = [participant.strip() for participant in def_participants]
    # add the 'Operator' to the list
    def_participants.append('Operator')

    # drop the duplicated participants
    def_participants_copy = def_participants.copy()
    def_participants = []
    # drop the duplicated participants
    for i in def_participants_copy: 
        if i not in def_participants: 
            def_participants.append(i) 
    def_participants = sorted(def_participants)
    
    return def_participants

all_participants = clean_participants_list(all_participants)
company_paticipants_list = clean_participants_list(company_paticipants_list)
other_paticipants_list = clean_participants_list(other_paticipants_list)


In [6]:
concat_df = pd.DataFrame()
for column in df_clean_na.columns:
    # identify all the rows in df with all_participants in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants)].index.tolist()
    # apply the both_participants_row_index to the df_clean_na['participants']
    concat_df[column] = df_clean_na[column]
    concat_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants else np.nan)
    # fill the NaN with the value of the previous row
    concat_df[f"participants_{column}"] = concat_df[f"participants_{column}"].fillna(method='ffill')

/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/3664898758.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  concat_df[column] = df_clean_na[column]
/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/3664898758.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  concat_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants else np.nan)


In [58]:
no_par_df = pd.DataFrame()
# identify the len before NaN of each column
for column in df_clean_na.columns:
    # exclude the row if no_par_df[column]==no_par_df[f"participants_{column}"]
    no_par_df = concat_df[concat_df[column] != concat_df[f"participants_{column}"]]

# use len(no_par_df.columns.to_list()) to write a for loop
model_df = pd.DataFrame()
for i in range(int(len(no_par_df.columns.to_list())/2)):
    tmp_df = pd.DataFrame()
    tmp_df = no_par_df.iloc[:,(i*2):(i*2)+2]
    # extract the index as column from the text
    tmp_df['file_name'] = tmp_df.columns.to_list()[0]
    # extract the date from the index column
    tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
    # change the date column to datetime
    tmp_df['date'] = pd.to_datetime(tmp_df['date'])
    # rename to be consistent with the column name
    tmp_df.columns = ["paragraph", "participants", "file_name","date"]
    # if the 'participants' column's value equals to any of the company_paticipants_list, other_paticipants_list, then set the value to 0
    tmp_df['company_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in company_paticipants_list else 0)
    tmp_df['other_paticipants_yes'] = tmp_df['participants'].apply(lambda x: 1 if x in other_paticipants_list else 0)
    # drop the row if the column "paragraph" is NaN
    tmp_df = tmp_df.dropna(subset=['paragraph'], how='all')
    tmp_df['company_name1']  = tmp_df['file_name'].apply(lambda x: x.split('_')[1])
    tmp_df['company_name2']  = tmp_df['file_name'].apply(lambda x: x.split('_')[2])
    tmp_df['company_name'] = tmp_df["company_name1"] + " " + tmp_df["company_name2"]
    # drop the 'company_name1' and 'company_name2' column
    tmp_df = tmp_df.drop(columns=['company_name1', 'company_name2'])
    model_df = model_df.append(tmp_df)

model_df

/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/2318456439.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['file_name'] = tmp_df.columns.to_list()[0]
/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/2318456439.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['date'] = tmp_df['file_name'].apply(lambda x: x.split('_')[0])
/var/folders/yp/gsl3sklx6bg6x689_qhhd7300000gn/T/ipykernel_55937/2318456439.py:17: SettingWithCopyWarning: 
A value is trying to be se

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name
1,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings
2,sure you're tired by now at this late hour but...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings
3,What I am going to explain today is only the f...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings
4,full fiscal year of 2012 and shareholder retur...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings
5,"NKSJ Group management plan announced today, Pr...",Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings
...,...,...,...,...,...,...,...
146,"profits should be clearly lower. However, in t...",Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-
147,"positive bottom line impacts from goodwill, ta...",Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-
148,course of the annual closing. I do not see why...,Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-
149,Thank you very much for listening. Nikolaus vo...,Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-


## Stock price

In [12]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import seaborn as sns

import matplotlib.pyplot as plt 

In [51]:
# dataframe company and its tickers
stock_list = [
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8766.T','8766.T', # this company has 3 file name
    '8630.T','8725.T','QBE.AX',
]
company_name = [
    # European (Re)Insurers
    'SCOR SE-','Munich Re-','Swiss Re','Hannover Rueck',
    # Japanese & Pacific
    'TOKIO MARINE', 'HCC INSURANCE-','MITSUI FUDOSAN-',
    'Sompo Holdings','MS-AD INSURANCE-','QBE INSURANCE-'
               ]
company_ticker_df = pd.DataFrame({'company_name':company_name,
                                  'ticker':stock_list})
company_ticker_df

,company_name,ticker
0,SCOR SE-,SCR.PA
1,Munich Re-,MUV2.DE
2,Swiss Re,SREN.SW
3,Hannover Rueck,HNR1.DE
4,TOKIO MARINE,8766.T
5,HCC INSURANCE-,8766.T
6,MITSUI FUDOSAN-,8766.T
7,Sompo Holdings,8630.T
8,MS-AD INSURANCE-,8725.T
9,QBE INSURANCE-,QBE.AX


In [110]:
# function for find price change
def price_change(aapl_df, n_day):
    before = []
    after = []
    for i in range(len(aapl_df)):
        # price on that date
        prc_tdy = aapl_df['Close'][i]

        if ((i >= n_day) & 
            (i < (len(aapl_df)-n_day))):
            # price change before n_day
            prc_before = aapl_df['Close'][i-n_day]
            prc_change_before = (prc_tdy - prc_before)/prc_before
            # price change after n_day
            prc_after = aapl_df['Close'][i+n_day]
            prc_change_after = (prc_after - prc_tdy)/prc_tdy
        else:
            prc_change_before = 0
            prc_change_after = 0
        # append into list
        before.append(prc_change_before)
        after.append(prc_change_after)
    return before, after

In [114]:
# tickers and its closing stock price
stock_df = pd.DataFrame()
for i in stock_list:
    aapl_df = yf.download(i, 
                      start='2010-12-01', # start 1 month before
                      end='2022-01-30', # end 1 month later
                      progress=False,)
    aapl_df = aapl_df.reset_index(drop=False)
    # add ticker name
    aapl_df['ticker']=i
    # add price change
    aapl_df['D-1'],aapl_df['D+1'] = price_change(aapl_df, 1)
    aapl_df['D-2'],aapl_df['D+2'] = price_change(aapl_df, 2)
    aapl_df['D-3'],aapl_df['D+3'] = price_change(aapl_df, 3)
    aapl_df['D-5'],aapl_df['D+5'] = price_change(aapl_df, 5)
    aapl_df['D-10'],aapl_df['D+10'] = price_change(aapl_df, 10)
    aapl_df['D-15'],aapl_df['D+15'] = price_change(aapl_df, 15)
    # append into one dataframe 
    stock_df = stock_df.append(aapl_df)
    
# drop redundancy columns
stock_price_df = stock_df.drop(columns=['Open', 'High','Low','Adj Close','Volume'])
stock_price_df = stock_price_df.rename(columns={'Date': 'date'})
# drop duplicated rows
stock_price_df = stock_price_df.drop_duplicates()
stock_price_df.head(5)

,date,Close,ticker,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,2010-12-01,18.190001,SCR.PA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-12-02,18.309999,SCR.PA,0.006597,-0.004369,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-12-03,18.230000,SCR.PA,-0.004369,-0.000549,0.002199,0.009600,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-12-06,18.219999,SCR.PA,-0.000549,0.010154,-0.004915,0.018386,0.001649,0.033480,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-12-07,18.405001,SCR.PA,0.010154,0.008150,0.009600,0.023092,0.005188,0.020103,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
# merged company tickers into big dataframe
prc_n_tkr_df = model_df.copy()
prc_n_tkr_df = prc_n_tkr_df.join(company_ticker_df.set_index('company_name'), 
                         on='company_name',
                         how='left'
                        )
#print(len(prc_n_tkr_df))
# merged stock price in
#merged_data= prc_n_tkr_df.merge(stock_price_df, on=["date","ticker"], how='left')
merged_data = prc_n_tkr_df.join(stock_price_df.set_index(["date","ticker"]), 
                         on=["date","ticker"],
                         how='left'
                        )
# drop na rows, which date in on Sat or Sun so it did not have the stock price
# merged_data.loc[merged_data['Close'].isnull()==True]
merged_data = merged_data.dropna()
merged_data

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name,ticker,Close,D-1,...,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
1,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.00,0.038436,...,0.064796,-0.050816,0.097796,-0.021330,0.140200,-0.012547,0.106176,-0.029486,0.117029,-0.000627
2,sure you're tired by now at this late hour but...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.00,0.038436,...,0.064796,-0.050816,0.097796,-0.021330,0.140200,-0.012547,0.106176,-0.029486,0.117029,-0.000627
3,What I am going to explain today is only the f...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.00,0.038436,...,0.064796,-0.050816,0.097796,-0.021330,0.140200,-0.012547,0.106176,-0.029486,0.117029,-0.000627
4,full fiscal year of 2012 and shareholder retur...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.00,0.038436,...,0.064796,-0.050816,0.097796,-0.021330,0.140200,-0.012547,0.106176,-0.029486,0.117029,-0.000627
5,"NKSJ Group management plan announced today, Pr...",Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.00,0.038436,...,0.064796,-0.050816,0.097796,-0.021330,0.140200,-0.012547,0.106176,-0.029486,0.117029,-0.000627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,"profits should be clearly lower. However, in t...",Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-,MUV2.DE,173.25,0.014939,...,0.020619,0.005772,0.025451,-0.007504,0.029412,-0.023665,0.011384,-0.047330,-0.006024,-0.058009
147,"positive bottom line impacts from goodwill, ta...",Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-,MUV2.DE,173.25,0.014939,...,0.020619,0.005772,0.025451,-0.007504,0.029412,-0.023665,0.011384,-0.047330,-0.006024,-0.058009
148,course of the annual closing. I do not see why...,Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-,MUV2.DE,173.25,0.014939,...,0.020619,0.005772,0.025451,-0.007504,0.029412,-0.023665,0.011384,-0.047330,-0.006024,-0.058009
149,Thank you very much for listening. Nikolaus vo...,Joerg Schneider,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD0...,2015-08-06,0,0,Munich Re-,MUV2.DE,173.25,0.014939,...,0.020619,0.005772,0.025451,-0.007504,0.029412,-0.023665,0.011384,-0.047330,-0.006024,-0.058009


# Sentiment analysis

In [23]:
model_df

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes
1,"Good morning, ladies and gentlemen. I welcome ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1
2,Conference Call on the Q3 2021 Financial Resul...,Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1
3,"being recorded. At this time, I would like to ...",Operator,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,1,1
4,Jean-Jacques Henchoz,Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1
5,"Well, good morning, everyone and welcome to ou...",Jean-Jacques Henchoz,20211104_Hannover_Rueck_SE-_Earnings_Call_2021...,2021-11-04,0,1
...,...,...,...,...,...,...
770,"Our ordinary income, that's very pleasing. And...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1
771,portfolio has been very reliable in terms of p...,Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1
772,"low interest rate environment, of course is ch...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1
773,"inflation risk side, again I think it's over -...",Karl Steinle,20211014_Hannover_Rueck_SE-_Shareholder_Mtg_Ca...,2021-10-14,0,1


In [26]:
# write a function to split the text by '.' in paragraph_split_df
p_to_s_split_df = model_df[['paragraph','participants']].copy()

def split_text(text):
    text = text.split(". ")
    return text
# apply the function to the paragraph_split_df
p_to_s_split_df['paragraph'] = p_to_s_split_df['paragraph'].apply(lambda x: split_text(x))
p_to_s_split_df

,paragraph,participants
1,"[Good morning, ladies and gentlemen, I welcome...",Operator
2,[Conference Call on the Q3 2021 Financial Resu...,Operator
3,"[being recorded, At this time, I would like to...",Operator
4,[Jean-Jacques Henchoz],Jean-Jacques Henchoz
5,"[Well, good morning, everyone and welcome to o...",Jean-Jacques Henchoz
...,...,...
770,"[Our ordinary income, that's very pleasing, An...",Karl Steinle
771,[portfolio has been very reliable in terms of ...,Karl Steinle
772,"[low interest rate environment, of course is c...",Karl Steinle
773,"[inflation risk side, again I think it's over ...",Karl Steinle


In [29]:
sentence_split_df = pd.DataFrame()
for i in range(len(p_to_s_split_df)):
    sentence_list = p_to_s_split_df['paragraph'].iloc[i]
    sentence_split_single_df = pd.DataFrame (sentence_list, columns = ['sentence'])
    sentence_split_single_df['participants'] = p_to_s_split_df['participants'].iloc[i]
    sentence_split_single_df['paragraph_num'] = i
    sentence_split_df = sentence_split_df.append(sentence_split_single_df, ignore_index=True)
# drop if the 'sentence' is empty
sentence_split_df = sentence_split_df.dropna(inplace=False)

sentence_split_df

,sentence,participants,paragraph_num
0,"Good morning, ladies and gentlemen",Operator,0
1,I welcome you to today's Hannover Re Internati...,Operator,0
2,Conference Call on the Q3 2021 Financial Results,Operator,1
3,"For your information, this conference is",Operator,1
4,being recorded,Operator,2
...,...,...,...
38575,On the,Karl Steinle,25616
38576,"inflation risk side, again I think it's over -...",Karl Steinle,25617
38577,Inflation drivers are monitored,Karl Steinle,25618
38578,We are monitoring them very closely,Karl Steinle,25618


In [34]:
# twitter_samples.fileids()
# documents
docs_negative = [(t, "neg") for t in twitter_samples.strings("negative_tweets.json")]
docs_positive = [(t, "pos") for t in twitter_samples.strings("positive_tweets.json")]
print("==========================================================")
print(f'There are {len(docs_negative)} negative sentences.')
print(f'There are {len(docs_positive)} positive sentences.')

# spliting dataset 
train_set = docs_negative[:3500] + docs_positive[:3500]
test_set = docs_negative[3500:4250] + docs_positive[3500:4250]
valid_set = docs_negative[4250:] + docs_positive[4250:]

# clean text
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    #text = text.str
    text = str(text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'#', '', text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)
            
    sentence = ' '.join(text_clean)
    
    return sentence

# categorical label
def cat_label(label):
    if label == 'neg':
        value = -1
    elif label == 'pos':
        value = 1
    return value 

# split for x and y 
def xy(dataset):
    df = pd.DataFrame(dataset, columns = ['text', 'label'])
    df['text_clean'] = df['text'].apply(lambda r: process_text(r))
    #df['categorical_label'] = df.label.factorize()[0]
    df['categorical_label'] = df['label'].apply(lambda r: cat_label(r))

    x = df.text_clean
    y = df.categorical_label

    return x, y

# dataframe
x_train, y_train = xy(train_set)
x_test, y_test = xy(test_set)
x_valid, y_valid = xy(valid_set)

## using the naive bayes classifier
model = Pipeline([
    ('bow',CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print("==========================================================")
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))
print(accuracy_score(y_pred,y_test))

# Apply into earnings call sentence
df_sentence = sentence_split_df.copy()
# df_sentence.head()

# drop participant columns as we dont need it
# df_sentence = df_sentence.drop(['participants'], axis=1)

# check NaN values
print("==========================================================")
print(df_sentence.isnull().sum())

# delete NaN rows
df_sentence = df_sentence.dropna()  

# clean text for sentiment analysis
df_sentence['text_clean'] = df_sentence['sentence'].apply(lambda r: process_text(r))
# df_sentence.head(5)

# making prediction
prediction = model.predict(df_sentence.text_clean)
prediction_label = np.array(['positive' if p==1 else 'negative' for p in prediction])
df_sentence['prediction_label'] = prediction_label
df_sentence['sentiment_score'] = prediction
# df_sentence.head()

print("==========================================================")
print(Counter(df_sentence['prediction_label']))

# df_sentence left columns with only 'sentence','participants','sentiment_score'
df_sentence = df_sentence[['sentence','participants','sentiment_score']]

sentence_split_df['sentiment_score']=df_sentence['sentiment_score']
sentence_split_df

There are 5000 negative sentences.
There are 5000 positive sentences.
[[593 235]
 [157 515]]
              precision    recall  f1-score   support

          -1       0.79      0.72      0.75       828
           1       0.69      0.77      0.72       672

    accuracy                           0.74      1500
   macro avg       0.74      0.74      0.74      1500
weighted avg       0.74      0.74      0.74      1500

0.7386666666666667
sentence         0
participants     0
paragraph_num    0
dtype: int64
Counter({'positive': 20059, 'negative': 18521})


,sentence,participants,paragraph_num,sentiment_score
0,"Good morning, ladies and gentlemen",Operator,0,1
1,I welcome you to today's Hannover Re Internati...,Operator,0,1
2,Conference Call on the Q3 2021 Financial Results,Operator,1,-1
3,"For your information, this conference is",Operator,1,-1
4,being recorded,Operator,2,1
...,...,...,...,...
38575,On the,Karl Steinle,25616,-1
38576,"inflation risk side, again I think it's over -...",Karl Steinle,25617,-1
38577,Inflation drivers are monitored,Karl Steinle,25618,-1
38578,We are monitoring them very closely,Karl Steinle,25618,-1


[1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1